In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

import os
from pathlib import Path

from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer

from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Fri Nov 21 02:57:18 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


# Define data directories

In [3]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [4]:
# # For testing: Run directory containing model 8.3 results for all
# # locations
# model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# # Results directory for model 8.3, for testing
# results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 8.3 - 8.7
artifact_model_number = '8.3'


In [5]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory
# with os.scandir(model_run_dir_8_7) as entries:
#     batch_run_dirs_8_7 = [entry.path for entry in entries]
batch_run_dirs_8_7 = loading.list_paths(model_run_dir_8_7)

# Select a model and filter for testing
batch_run_dirs = batch_run_dirs_8_7#[:2]

batch_run_dirs

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_42_37',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_07_09_05_46',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_10_07_44_18',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_43_53',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_13_00_54',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_16_28_26',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_07_13_14_27',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/result

# Create an AlzheimersResultsProcessor object to process results

In [6]:
results = AlzheimersResultsProcessor('8.3', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [7]:
results.initial_simulation_population

2000000

In [8]:
results.locations

['United States of America',
 'Brazil',
 'China',
 'Germany',
 'Israel',
 'Japan',
 'Spain',
 'Sweden',
 'Taiwan (Province of China)',
 'United Kingdom']

In [9]:
results.draws

[457,
 169,
 323,
 258,
 446,
 89,
 41,
 113,
 356,
 273,
 158,
 24,
 265,
 317,
 177,
 199,
 392,
 346,
 480,
 262,
 460,
 499,
 2,
 13,
 46]

# Load population data and calculate model scale

In [10]:
with Timer():
    results.load_population_data()
results.ops.describe(results.model_scale)

Elapsed time: 0:00:14.341348


,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Brazil,25.0,1.116473,0.102910,0.886671,1.059915,1.119741,1.145108,1.386433
China,25.0,0.123020,0.007061,0.106308,0.120095,0.123499,0.125744,0.143724
Germany,25.0,0.922878,0.055810,0.854854,0.876986,0.914862,0.983316,1.012529
Israel,25.0,41.883212,2.979755,37.896818,39.739917,40.651231,43.913069,48.068295
Japan,25.0,0.586225,0.042318,0.511002,0.557443,0.585295,0.611764,0.694869
Spain,25.0,3.192170,0.169602,2.875882,3.073433,3.190730,3.289187,3.574185
Sweden,25.0,12.993787,0.944507,11.361366,12.398862,12.890094,13.665418,14.830542
Taiwan (Province of China),25.0,8.853801,0.468546,8.050507,8.487516,8.855923,9.154749,9.783891
United Kingdom,25.0,2.057251,0.093390,1.891726,1.987250,2.042450,2.134812,2.215977


In [11]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,2,1.142971e+07,25_to_29
1,United States of America,Female,2025,13,1.100320e+07,25_to_29
2,United States of America,Female,2025,24,1.143598e+07,25_to_29
3,United States of America,Female,2025,41,1.126842e+07,25_to_29
4,United States of America,Female,2025,46,1.114181e+07,25_to_29
...,...,...,...,...,...,...
569995,United Kingdom,Male,2100,446,1.254920e+05,95_plus
569996,United Kingdom,Male,2100,457,1.446562e+05,95_plus
569997,United Kingdom,Male,2100,460,1.251287e+05,95_plus
569998,United Kingdom,Male,2100,480,1.243646e+05,95_plus


# Deaths and averted deaths

In [12]:
with Timer():
    results.load_measure_from_batch_runs('deaths', filters=loading.FINAL_RESULTS_FILTERS['deaths'])

print(len(results.data.deaths), 'rows')
results.data.deaths.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '=', 'alzheimers_disease_state')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing_and_treatment,Female,0.0,Japan


In [13]:
deaths_prepped = results.process_deaths(results.data.deaths)
deaths_prepped

47.885504 MB measure
15.965504 MB minuend
31.925504 MB subtrahend
10.269531 MB minuend re-indexed
20.529531 MB subtrahend re-indexed
20.529591 MB difference
36.485636 MB difference with reset index
37.625961 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location,metric
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,baseline,Female,0.0,Japan,Number
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,baseline,Male,0.0,Japan,Number
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing,Female,0.0,Japan,Number
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing,Male,0.0,Japan,Number
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing_and_treatment,Female,0.0,Japan,Number
...,...,...,...,...,...,...,...,...,...,...,...,...
2849995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,13,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Male,-383.0,Brazil,Number
2849996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,46,Averted Deaths Associated with AD,bbbm_testing,Female,0.0,Brazil,Number
2849997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,46,Averted Deaths Associated with AD,bbbm_testing,Male,0.0,Brazil,Number
2849998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,46,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Female,-636.0,Brazil,Number


In [14]:
deaths_output = results.summarize_and_beautify(deaths_prepped, 'entity')
deaths_output

Timer SummarizingTimer started
2025-11-21 02:58:31.915033
2025-11-21 02:58:46.866166
2025-11-21 03:04:02.014914
Elapsed time: 330.1116 seconds


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,25_to_29,Female,AD Dementia,Reference,Deaths Associated with AD,Number,0.000000,0.000000,0.000000
1,2025,Brazil,25_to_29,Male,AD Dementia,Reference,Deaths Associated with AD,Number,0.000000,0.000000,0.000000
2,2025,Brazil,25_to_29,Female,AD Dementia,BBBM Testing Only,Deaths Associated with AD,Number,0.000000,0.000000,0.000000
3,2025,Brazil,25_to_29,Male,AD Dementia,BBBM Testing Only,Deaths Associated with AD,Number,0.000000,0.000000,0.000000
4,2025,Brazil,25_to_29,Female,AD Dementia,BBBM Testing and Treatment,Deaths Associated with AD,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
227995,2100,United States of America,95_plus,Male,AD Dementia,BBBM Testing and Treatment,Averted Deaths Associated with AD,Number,-54.041261,-86.985093,-32.639751
227996,2100,United States of America,95_plus,Female,AD Dementia,BBBM Testing Only,Averted Deaths Associated with AD,Rate,0.000000,0.000000,0.000000
227997,2100,United States of America,95_plus,Male,AD Dementia,BBBM Testing Only,Averted Deaths Associated with AD,Rate,0.000000,0.000000,0.000000
227998,2100,United States of America,95_plus,Female,AD Dementia,BBBM Testing and Treatment,Averted Deaths Associated with AD,Rate,-0.000157,-0.000242,-0.000099


In [15]:
deaths_output.to_csv(output_dir / "deaths.csv", index=False)

# DALYs and averted DALYs

In [16]:
filename = 'ylls'
short_name = filename

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '=', 'alzheimers_disease_state')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers

4.109634 MB after aggregating seeds and converting dtypes
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_17_16_49/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/mnt/te

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,ylls,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,ylls,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,ylls,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,ylls,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,ylls,bbbm_testing_and_treatment,Female,0.0,Japan


In [17]:
filename = 'ylds'
short_name = filename

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '==', 'alzheimers_disease_and_other_dementias')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium

12.934565 MB after aggregating seeds and converting dtypes
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_10_07_44_18/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/mnt/t

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,ylds,baseline,Female,alzheimers_blood_based_biomarker_state,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,ylds,baseline,Female,alzheimers_mild_cognitive_impairment_state,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,ylds,baseline,Female,alzheimers_disease_state,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,ylds,baseline,Male,alzheimers_blood_based_biomarker_state,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,ylds,baseline,Male,alzheimers_mild_cognitive_impairment_state,0.0,Japan


In [18]:
dalys = results.process_dalys(results.data.ylls, results.data.ylds)
print_memory_usage(dalys)
print(len(dalys), 'rows')
dalys.tail()

97.479395 MB measure
46.179263 MB minuend
92.349263 MB subtrahend
32.499984 MB minuend re-indexed
64.989984 MB subtrahend re-indexed
64.990044 MB difference
64.989395 MB difference with reset index
68.40972 MB final difference
171.009674 MB 
8550000 rows


,age_group,artifact_path,entity,entity_type,event_year,measure,sex,sub_entity,input_draw,scenario,location,value,metric
8549995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Female,alzheimers_disease_state,46,bbbm_testing_and_treatment,Brazil,-6045.984550,Number
8549996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Female,alzheimers_mild_cognitive_impairment_state,46,bbbm_testing_and_treatment,Brazil,-38.924598,Number
8549997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Male,alzheimers_blood_based_biomarker_state,46,bbbm_testing_and_treatment,Brazil,0.000000,Number
8549998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Male,alzheimers_disease_state,46,bbbm_testing_and_treatment,Brazil,-2025.302592,Number
8549999,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Male,alzheimers_mild_cognitive_impairment_state,46,bbbm_testing_and_treatment,Brazil,-12.046432,Number


In [19]:
# 5m 22s to run on first 3 batches of 8.4 (10 draws)
with Timer():
    dalys_output = results.summarize_and_beautify(dalys)
print_memory_usage(dalys_output)
print(len(dalys_output), 'rows')
dalys_output.tail()

Timer SummarizingTimer started
2025-11-21 03:09:48.607741


2025-11-21 03:10:45.705363
2025-11-21 03:27:22.270823
Elapsed time: 1053.6887 seconds
Elapsed time: 0:17:33.689092
21.894738 MB 
684000 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
683995,2100,United States of America,95_plus,Female,AD Dementia,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,-0.001557,-0.002322,-0.000983
683996,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,-0.000018,-0.000032,-0.000007
683997,2100,United States of America,95_plus,Male,Preclinical AD,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,0.000000,0.000000,0.000000
683998,2100,United States of America,95_plus,Male,AD Dementia,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,-0.000876,-0.001326,-0.000534
683999,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,-0.000010,-0.000021,-0.000004


In [20]:
dalys_output.to_csv(output_dir / 'dalys.csv', index=False)

# Prevalence

In [21]:
filename = 'person_time_alzheimers_disease_and_other_dementias'
short_name = 'person_time_ad'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/m

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,treatment,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,susceptible_to_treatment,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,waiting_for_treatment,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,full_effect_long,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,full_effect_short,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,waning_effect_long,0.0,Japan


In [22]:
prevalence_counts = results.process_prevalence_counts(results.data.person_time_ad)
print_memory_usage(prevalence_counts)
print(len(prevalence_counts), 'rows')
prevalence_counts.tail()

102.609607 MB 
5130000 rows


,age_group,artifact_path,entity,entity_type,event_year,input_draw,location,measure,scenario,sex,sub_entity,value,metric
5129995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Female,alzheimers_mild_cognitive_impairment_state,90833.571526,Number
5129996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Female,alzheimers_disease_state,206003.071869,Number
5129997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_blood_based_biomarker_state,59830.039699,Number
5129998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_mild_cognitive_impairment_state,46955.252567,Number
5129999,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_disease_state,91416.071184,Number


In [23]:
with Timer():
    prevalence_output = results.summarize_and_beautify(prevalence_counts)
print_memory_usage(prevalence_output)
print(len(prevalence_output), 'rows')
prevalence_output.tail()

Timer SummarizingTimer started
2025-11-21 04:00:35.069854


2025-11-21 04:01:07.297152
2025-11-21 04:11:22.333382
Elapsed time: 647.2805 seconds
Elapsed time: 0:10:47.281013
13.139435 MB 
410400 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
410395,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing and Treatment,Prevalence,Rate,0.061993,0.050527,0.073789
410396,2100,United States of America,95_plus,Female,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.110292,0.090575,0.130764
410397,2100,United States of America,95_plus,Male,Preclinical AD,BBBM Testing and Treatment,Prevalence,Rate,0.054555,0.031226,0.075501
410398,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,Prevalence,Rate,0.045778,0.037777,0.053021
410399,2100,United States of America,95_plus,Male,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.076582,0.063279,0.087289


In [24]:
# Save prevalence data
prevalence_output.to_csv(output_dir / 'prevalence.csv', index=False)

# Incidence

In [25]:
filename = 'counts_new_simulants'
short_name = 'incidence_bbbm'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/m

3.493437 MB after aggregating seeds and converting dtypes
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_17_16_49/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/mnt/te

,age_group,artifact_path,event_year,input_draw,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,bbbm_testing_and_treatment,Female,0.0,Japan


In [26]:
filename = 'transition_count_alzheimers_disease_and_other_dementias'
short_name = 'transitions_ad'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/m

81.266159 MB after aggregating seeds and converting dtypes
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_42_37/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/mnt/t

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,treatment,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,transition_count,baseline,Female,alzheimers_blood_based_biomarker_state_to_alzh...,susceptible_to_treatment,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,transition_count,baseline,Female,alzheimers_blood_based_biomarker_state_to_alzh...,waiting_for_treatment,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,transition_count,baseline,Female,alzheimers_blood_based_biomarker_state_to_alzh...,full_effect_long,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,transition_count,baseline,Female,alzheimers_blood_based_biomarker_state_to_alzh...,full_effect_short,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,transition_count,baseline,Female,alzheimers_blood_based_biomarker_state_to_alzh...,waning_effect_long,0.0,Japan


In [27]:
incidence_counts = results.process_incidence(
    results.data.incidence_bbbm, results.data.transitions_ad)
print(len(incidence_counts), 'rows')
print_memory_usage(incidence_counts)
incidence_counts

677.166959 MB raw transitions
71.825967 MB after marginalizing treatment
711.365967 MB after state extraction
['alzheimers_mild_cognitive_impairment_state' 'alzheimers_disease_state']
5130000 rows
133.389101 MB 


,age_group,artifact_path,event_year,input_draw,scenario,sex,value,location,to_state,measure,metric
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,baseline,Female,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,baseline,Male,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,bbbm_testing,Female,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,bbbm_testing,Male,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,169,bbbm_testing_and_treatment,Female,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number
...,...,...,...,...,...,...,...,...,...,...,...
3419995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,499,bbbm_testing,Male,11043.0,Brazil,alzheimers_disease_state,Incidence,Number
3419996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,499,bbbm_testing_and_treatment,Female,15678.0,Brazil,alzheimers_mild_cognitive_impairment_state,Incidence,Number
3419997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,499,bbbm_testing_and_treatment,Female,21921.0,Brazil,alzheimers_disease_state,Incidence,Number
3419998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,499,bbbm_testing_and_treatment,Male,9483.0,Brazil,alzheimers_mild_cognitive_impairment_state,Incidence,Number


In [28]:
with Timer():
    incidence_output = results.summarize_and_beautify(incidence_counts, 'to_state')
print_memory_usage(incidence_output)
print(len(incidence_output), 'rows')
incidence_output.tail()

Timer SummarizingTimer started
2025-11-21 04:32:00.272839


2025-11-21 04:32:24.306590
2025-11-21 04:40:50.444810
Elapsed time: 530.1844 seconds
Elapsed time: 0:08:50.184596
13.139434 MB 
410400 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
410395,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing and Treatment,Incidence,Rate,0.010858,0.006874,0.014800
410396,2100,United States of America,95_plus,Female,AD Dementia,BBBM Testing and Treatment,Incidence,Rate,0.013954,0.011423,0.016665
410397,2100,United States of America,95_plus,Male,Preclinical AD,BBBM Testing and Treatment,Incidence,Rate,0.000000,0.000000,0.000000
410398,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,Incidence,Rate,0.008126,0.005118,0.010830
410399,2100,United States of America,95_plus,Male,AD Dementia,BBBM Testing and Treatment,Incidence,Rate,0.010238,0.008470,0.011953


In [29]:
# Save incidence results
incidence_output.to_csv(output_dir / 'incidence.csv', index=False)

In [30]:
!date

Fri Nov 21 04:40:56 PST 2025
